#  <center> Taller  de Aprendizaje Automático </center>
##  <center> Taller 9: *Natural Language Processing* (NLP)  </center>

## Introducción

La siguiente actividad propone el abordaje de un problema de procesamiento de lenguaje natural (NLP) utilizando herramientas de *embedding* y modelos RNN. El conjunto de datos que se utilizará es IMDb, el cual corresponde a un problema de clasificación donde se tienen 50000 criticas de películas (35000 de *train* y 15000 de *test*), y se quiere estimar si éstas son críticas positivas (1) o negativas (0). 

La propuesta consiste en entender y reproducir los pasos de la sección *Sentiment Analysis* para los datos **sin procesar**, agregando algunas variantes como mitigar el sobreajuste y entender la herramienta *embeddings*.

En este Taller también se introduce la biblioteca *Streamlit*, utilizada para desarrollar prototipos de aplicaciones web de aprendizaje automático. Aquellos que así lo deseen, podrán generar de manera sencilla una aplicación web que clasifique las críticas proporcionadas por los usuarios.


## Objetivos


*   Aplicar modelos basados en RNN a un problema de NLP.
*   Trabajar con embeddings para secuencias de texto, en particular embeddings preentrenados.
*   Utilizar herramientas para la visualización de embeddings.
*  (Opcional, no evaluado) Desarrollar una aplicación web que clasifique críticas proporcionadas por los usuarios.

## Formas de trabajo

### Opción 1: Trabajar localmente


Descargar los datos en su máquina personal y trabajar en su propio ambiente de desarrollo.

`conda activate TAA-py38`              
`jupyter-notebook`    

Los paquetes faltantes se pueden instalar desde el notebook haciendo:     
` !pip install paquete_faltante` 

### Opción 2:  Trabajar en *Colab*. 

<table align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/TAA-fing/TAA-2022/blob/main/talleres/taller9_NLP.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Ejecutar en Google Colab</a>
  </td>
</table>

Se puede trabajar en Google Colab. Para ello es necesario contar con una cuenta de **google drive** y ejecutar un notebook almacenado en dicha cuenta. De lo contrario, no se conservarán los cambios realizados en la sesión. En caso de ya contar con una cuenta, se puede abrir el notebook y luego ir a `Archivo-->Guardar una copia en drive`. 

La siguiente celda realiza la configuración necesaria para obtener datos desde la plataforma Kaggle. Le solicitará que suba el archivo *kaggle.json* asociado a su cuenta.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from google.colab import files

# El siguiente archivo solicitado es para habilitar la API de Kaggle en el entorno que está trabajando.
# Este archivo se descarga entrando a su perfíl de Kaggle, en la sección API, presionando donde dice: Create New API Token

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

#Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

# Parte 1: Análisis y preprocesamiento de datos


Se utilizará el conjunto de IMDb provisto por Kaggle. Se tienen 50000 criticas de películas que al igual que en el *Taller 2* se utilizarán 35000 para *train* y 15000 para *test*.

*   Ejecutar la siguiente celda para descargar el conjunto y verificar que los conjuntos tienen la cantidad de instancias esperadas.

In [ ]:
# Descarga la base IMDb de Kaggle
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [ ]:
import zipfile
import pandas as pd
import tensorflow as tf

# Se descomprime el archivo descargado
with zipfile.ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zip_ref:
    zip_ref.extractall('')

# Se levanta como pandas DataFrame
data_file = 'IMDB Dataset.csv'
data = pd.read_csv(data_file)

#Separación de Conjuntos
N=35000
X_train = data.loc[:N-1, 'review'].values
y_train = data.loc[:N-1, 'sentiment'].values == 'positive' 
X_test = data.loc[N:, 'review'].values
y_test = data.loc[N:, 'sentiment'].values == 'positive'

# Armado de los Tensorflow Datasets
dataset_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))

#Verificación
print('Train Size:', len(dataset_train),'Test Size:', len(dataset_test))

*   Del conjunto de entrenamiento visualizar tanto una crítica positiva como una negativa. Se sugiere ir a los Notebooks del *Capítulo 16*. Puede ser útil el uso del método *.skip()*.

*   Reservar unas 5000 críticas de los datos de entrenamiento para validación. 

* Siguiendo el ejemplo del libro genere una función para preprocesar las reseñas, y una función que genere la tabla del vocabulario utilizando solamente los datos de *train*.

In [ ]:
def build_vocabulary(datasets, batch_size=32, vocab_size=10000, num_oov_buckets=1000):
  
  '''
  Función que dado un tensorflow dataset (datasets) con críticas, calcula el número de ocurrencias 
  de las palabras y genera un vocabulario con las vocab_size palabras más frecuentes. Una vez generado 
  ese vocabulario, la función devuelve las palabras del vocabulario (words) con sus respectivos índices 
  (word_ids) y una tabla de vocabulario que admite num_oov_buckets palabras fuera del vocabulario. 

  Entrada: 

    datasets: Tensorflow Dataset con las críticas. 
    batch_size: Tamaño de los batches a preprocesar.
    vocab_size: Tamaño del Vocabulario
    num_oov_buckets: Cantidad de palabras fuera del vocabulario (Out-of-Vocabulary bucket)

  Salida:
    table: Tabla del Vocabulario
    words: Palabras del Vocabulario
    word_ids: Indices de las palabras en el Vocabulario

  '''
  
  #Solución
  
  return table, words, word_ids

* Genere una función que a partir de la reseña y la tabla del vocabulario genere un vector para entrenar. 

* Preprocese los conjuntos 

* ¿Cuáles son las 15 palabras más frecuentes en los datos de *train*? De acuerdo con lo observado, ¿considera que se podría mejorar el preprocesamiento?

Respuesta: 

*   Una vez preprocesados los datos ¿cuáles son los largos de secuencias para los primeros *batches*? ¿Es un problema a resolver que todos tengan largos distintos? ¿Por qué?.

Respuesta: 

# Parte 2: *Embedding*

* Cree y entrene el modelo que aparece al final de la sección *Sentiment Analysis* y previo a la sección *Masking* del capítulo 16.

* Observe que el modelo se sobreajusta a los datos. Utilice alguna técnica vista para regularizar. 

* El modelo cuenta con una capa de entrada de *embedding* la cual abarca la mayoría de los parámetros entrenables. En este caso un *embedding* es un vector entrenable que representa una palabra en nuevo espacio cuyo tamaño es un hiperparámetro. La concatenación de estos vectores conforma la matriz de *embedding*, donde su cantidad de filas corresponde a la suma del tamaño del vocabulario más la cantidad reservada para los *out-of-vocabulary* (vocab_size + num_oov_buckets), y la cantidad de columnas a las dimensiones de los vectores (*embed_size*). Al igual que una matriz de pesos, ésta se inicializa de forma aleatoria, y actualiza sus valores para cada *step* de entrenamiento.

  *   ¿Cuál es la ventaja de utilizar una capa de *embedding*? (Ver la sección *Encoding Categorical Features Using Embeddings* del capítulo 13 del libro.)
  *   Visualizar una representación del espacio de *embedding* utilizando *Comet*.

  Para este último punto se recomienda seguir el siguiente ejemplo: [logging-embeddings](https://www.comet.ml/docs/user-interface/embeddings/#logging-embeddings). Tener en cuenta que el parámetro *labels* de la función tiene que ser un *array* de *strings*, por lo cual si las palabras están codificadas con *utf-8* es necesario decodificarlas: 


```
decoder = np.vectorize(lambda x: x.decode('UTF-8'))
words_dec = decoder(words.numpy())
 ```

*   Para su mejor modelo: visualizar a qué distancias se encuentran las palabras unas de otras tanto en la representación a baja dimensión como en el espacio de *embedding*. Sobre todo probar con adjetivos positivos (*wonderful*, *excellent*, etc.) y negativos (*ugly*, *boring*, etc.) comparando los resultados. ¿Qué logra observar?.

# Parte 3: *Embedding Preentrenado*

Una de las técnicas para mejorar el desempeño en este tipo de problemas es utilizar *embeddings* ya entrenados. 
Siguiendo el ejemplo [Using pre-trained word embeddings](https://keras.io/examples/nlp/pretrained_word_embeddings/):

*   Descargar el *embedding* preentrenado [GloVE](https://nlp.stanford.edu/projects/glove/) que aparece en la sección *Load pre-trained word embeddings*.

* Preparar la nueva matriz de *embedding*. ¿Cuántas palabras del conjunto de entrenamiento se encuentran en el vocabulario de GloVE? ¿Cuántas no?.

* Entrenar el modelo con la nueva matriz de *embedding* de manera que los valores de ésta se mantengan fijos (ver parámetro en la capa de *embedding*).

* Continue el entrenamiento ahora con el *embedding* entrenable. Modifique el *learning rate*.

*   Comparar con los modelos anteriores en cuanto al desempeño, la cantidad de parámetros y el tiempo de entrenamiento.


Respuesta: 

*   Visualizar cómo es el espacio de *embedding*. ¿Qué diferencias puede observar con respecto al anterior?


Respuesta: 

# Parte 4: Mejoras en los Modelos (*Opcional*)

Se sugieren algunas líneas que podrían mejorar los desempeños obtenidos en las partes anteriores: 

* Variar la cantidad de caracteres que se mantienen de la reseña así como el tamaño del vocabulario con el que se entrena. 
* Modificar la función de preprocesado.
* Modificar las neuronas recurrentes. 

Piense por qué tiene sentido explorar estas estrategias y pruebe alguna que considere relevante. 



# Parte 5: Desarrollo de una aplicación web (*Opcional, no evaluado*)

En esta parte veremos cómo generar una aplicación web sencilla que permita mostrar el funcionamiento de un modelo que hayamos entrenado. Para ello utilizaremos la biblioteca [Streamit](https://streamlit.io/).

## Streamlit
Streamlit es una biblioteca de código abierto escrita en Python que permite crear y compartir aplicaciones web que usan algoritmos de aprendizaje automático. En la [documentación](https://docs.streamlit.io/) encontrará información sobre cómo instalar y crear aplicaciones utilizando la biblioteca. El flujo de trabajo básico consta de los siguientes pasos:   

1. Instalación   
2. Desarrollo de la aplicación   
3. Despliegue de la aplicación

### Instalación

En la mayoría de los casos, la biblioteca debería quedar instalada luego de crear un ambiente virtual (por ejemplo de conda) y hacer:   

`pip install streamlit`  

Puede verificar que la instalación sea correcta haciendo:    

`streamlit hello`

Puede ver los detalles de instalación en https://docs.streamlit.io/library/get-started/installation

### Desarrollo de la aplicación  

Se sugiere desarrollar la aplicación partiendo de un ejemplo que clasifica críticas de cine utilizando un modelo entrenado con las técnicas vistas en el Taller 2. Dicho ejemplo puede verse en funcionamiento [acá](https://share.streamlit.io/taa-fing/taa-2022/main/apps/movie_review_app/movie_review_app.py). La siguiente celda descarga el código fuente y lo descomprime. 

In [ ]:
!wget iie.fing.edu.uy/~carbajal/movie_review/apps.zip
!unzip apps.zip

Copie el archivo *movie_review_app.py*, modifique el nombre, y realice las siguientes modificaciones (además de las de diseño que crea conveniente):   

**Cambio de Modelo:**  Para hacer inferencia fuera de este *Notebook* será necesario contar con el vocabulario y el modelo entrenado. El modelo se puede guardar con alguna de las técnicas vistas en el curso. Para el vocabulario, existen distintas formas, se otorga un código que almacena las palabras del diccionario en un archivo de *numpy*. Con estas palabras luego será necesario generar la *Tabla* del vocabulario nuevamente. 

In [ ]:
# PATH a la dirección donde se almacenan las palabras del vocabulario. 
path_to_words = '/content/.../words.npy'

# Guardo las palabras 
np.save(path_to_words, words.numpy(), allow_pickle=True)   

#Levanto las palabras
words = np.load(path_to_words, allow_pickle=True)

**Modificación del pipeline de inferencia:**  El objetivo es generar una función que a partir de una única reseña prediga si la reseña es *positiva*(1) o *negativa*(0). Para ello se brinda una función a completar 

In [ ]:
# Pipeline funtion to make inference 
def pipeline_inference (review, words, model): 

  '''
  Función que prepara una review aislada y hace inferencia con el modelo. 

  Entradas: 
    review: String con la review a hacer inferencia
    words: Arreglo de numpy que contiene las palabras del vocabulario
    model: Modelo entrenado con el que se realiza inferencia

  Salida: 

    pred: Predicción del modelo

  '''
  
  # Parametros que haya utiliazado en el preprocesado y en la creación del vocabulario
  n_characters= ?
  vocab_size= ? 
  num_oov_buckets= ?

  # Código para crear la tabla del vocabulario a partir de las palabras del diccionario
  # ...
  
  # Código para codificar la reseña ingresada
  # ...

  # Código para convertir la reseña codificada a un tensor. Se sugiere utilizar tf.convert_to_tensor(). 
  # ...

  # Cadena de Preprocesado de la reseña tal como lo hizo en el entrenamiento.
  # ...

  # Códifcar la reseña utilizando la tabla del vocabulario
  # ...

  # Agregar una dimension al tensor correspondiente al batch. Se sugiere utilizar la función tf.expand_dims()
  # ...

  # Código para hacer inferencia con el modelo
  # ...
  
  return pred

* Las siguientes celdas prueban la función. Verifique que funciona correctamente.

In [ ]:
# Levanto el modelo
checkpoint_filepath = '/content/...'

model_loaded = keras.models.load_model(checkpoint_filepath)

In [ ]:
# Ejemplo de Reseña Negativa
review = 'This movie is really boring. I do not recommend it.'

pred = pipeline_inference(review, words, model_loaded)
pred

In [ ]:
# Ejemplo de Reseña Positiva
review = 'This movie is wonderful. I love it.'

pred = pipeline_inference(review, words, model_loaded)
pred

### Correr la aplicación localmente

Una vez realizadas las modificaciones en el archivo principal, cree un directorio donde guardar los archivos de su aplicación. Guarde allí el modelo, el archivo tipo *numpy* con las palabras y su archivo principal (Ej. *movie_review_app.py*).

* Una vez modificado el código, puede probarlo localmente. Para ello ejecute el siguiente comando, sustituyendo *movie_review_app.py* por el nombre de su archivo principal.


`!streamlit run apps/movie_review_app/movie_review_app.py`    

* También puede correrlo de forma local desde Colab con el comando: 

`!streamlit run apps/movie_review_app/movie_review_app.py & npx localtunnel --port 80`    

### Despliegue de la aplicación

Una vez que la app fue desarrollada es posible compartirla para que otros puedan probarla. Para ello es necesario:  

1. Contar con una cuenta de [Stramlit Cloud](https://docs.streamlit.io/streamlit-cloud/get-started#sign-up-for-streamlit-cloud) y un repositorio de GitHub donde almacenar el código. 
2.  Subir al repositorio  el código y los datos necesarios para correrlo. 
3. [Conectar la cuenta de Streamlit Cloud con la del repositorio](https://docs.streamlit.io/streamlit-cloud/get-started#connect-your-github-account).     
4. [Publicar la app](https://docs.streamlit.io/streamlit-cloud/get-started/deploy-an-app)

*Comentario:* Puede que sea necesario agregar en el repositorio un archivo *requirements.txt* donde deba especificar las librerías utilizas en el archivo *main.py*.